# Example functions and class

In [71]:
def func(param1=True, param2: str = 'default val'):
    """Description of func with docstring groups style (Googledoc).

    :param param1: descr of param1 that has True for default value
    :param param2: descr of param2 (Default value = 'default val')
    :type param2: str
    :returns: some value
    :raises keyError: raises key exception
    :raises TypeError: raises type exception

    """
    pass

class A:
    """ 
    a class without any interest
    """
    def method(self, param1, param2=None) -> int:
        """

        :param param1:
        :param param2:  (Default value = None)
        :rtype: int

        """
        pass

## Get docstring from function or class

In [72]:
print(func.__doc__)

Description of func with docstring groups style (Googledoc).

    :param param1: descr of param1 that has True for default value
    :param param2: descr of param2 (Default value = 'default val')
    :type param2: str
    :returns: some value
    :raises keyError: raises key exception
    :raises TypeError: raises type exception

    


# Taken from openstack

In [73]:
# https://github.com/openstack/rally

In [74]:
# Copyright 2015: Mirantis Inc.
# All Rights Reserved.
#
#    Licensed under the Apache License, Version 2.0 (the "License"); you may
#    not use this file except in compliance with the License. You may obtain
#    a copy of the License at
#
#         http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS, WITHOUT
#    WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the
#    License for the specific language governing permissions and limitations
#    under the License.

import re
import sys

PARAM_OR_RETURNS_REGEX = re.compile(":(?:param|returns)")
RETURNS_REGEX = re.compile(":returns: (?P<doc>.*)", re.S)
PARAM_REGEX = re.compile(":param (?P<name>[\*\w]+): (?P<doc>.*?)"
                         "(?:(?=:param)|(?=:return)|(?=:raises)|\Z)", re.S)


def trim(docstring):
    """trim function from PEP-257"""
    if not docstring:
        return ""
    # Convert tabs to spaces (following the normal Python rules)
    # and split into a list of lines:
    lines = docstring.expandtabs().splitlines()
    # Determine minimum indentation (first line doesn't count):
    indent = sys.maxsize
    for line in lines[1:]:
        stripped = line.lstrip()
        if stripped:
            indent = min(indent, len(line) - len(stripped))
    # Remove indentation (first line is special):
    trimmed = [lines[0].strip()]
    if indent < sys.maxsize:
        for line in lines[1:]:
            trimmed.append(line[indent:].rstrip())
    # Strip off trailing and leading blank lines:
    while trimmed and not trimmed[-1]:
        trimmed.pop()
    while trimmed and not trimmed[0]:
        trimmed.pop(0)

    # Current code/unittests expects a line return at
    # end of multiline docstrings
    # workaround expected behavior from unittests
    if "\n" in docstring:
        trimmed.append("")

    # Return a single string:
    return "\n".join(trimmed)


def reindent(string):
    return "\n".join(l.strip() for l in string.strip().split("\n"))


def parse_docstring(docstring):
    """Parse the docstring into its components.
    :returns: a dictionary of form
              {
                  "short_description": ...,
                  "long_description": ...,
                  "params": [{"name": ..., "doc": ...}, ...],
                  "returns": ...
              }
    """

    short_description = long_description = returns = ""
    params = []

    if docstring:
        docstring = trim(docstring)

        lines = docstring.split("\n", 1)
        short_description = lines[0]

        if len(lines) > 1:
            long_description = lines[1].strip()

            params_returns_desc = None

            match = PARAM_OR_RETURNS_REGEX.search(long_description)
            if match:
                long_desc_end = match.start()
                params_returns_desc = long_description[long_desc_end:].strip()
                long_description = long_description[:long_desc_end].rstrip()

            if params_returns_desc:
                params = [
                    {"name": name, "doc": trim(doc)}
                    for name, doc in PARAM_REGEX.findall(params_returns_desc)
                ]

                match = RETURNS_REGEX.search(params_returns_desc)
                if match:
                    returns = reindent(match.group("doc"))

    return {
        "short_description": short_description,
        "long_description": long_description,
        "params": params,
        "returns": returns
    }


class InfoMixin(object):

    @classmethod
    def _get_doc(cls):
        """Return documentary of class
        By default it returns docstring of class, but it can be overridden
        for example for cases like merging own docstring with parent
        """
        return cls.__doc__

    @classmethod
    def get_info(cls):
        doc = parse_docstring(cls._get_doc())

        return {
            "name": cls.get_name(),
            "platform": cls.get_platform(),
            "platform": cls.get_platform(),
            "module": cls.__module__,
            "title": doc["short_description"],
            "description": doc["long_description"],
            "parameters": doc["params"],
            "schema": getattr(cls, "CONFIG_SCHEMA", None),
            "returns": doc["returns"]
        }

In [75]:
docstring = func.__doc__
parse_docstring(docstring)

{'short_description': 'Description of func with docstring groups style (Googledoc).',
 'long_description': '',
 'params': [{'name': 'param1',
   'doc': 'descr of param1 that has True for default value\n'},
  {'name': 'param2',
   'doc': "descr of param2 (Default value = 'default val')\n:type param2: str\n"}],
 'returns': 'some value\n:raises keyError: raises key exception\n:raises TypeError: raises type exception'}

In [76]:
import inspect

In [77]:
inspect.getdoc(func)

"Description of func with docstring groups style (Googledoc).\n\n:param param1: descr of param1 that has True for default value\n:param param2: descr of param2 (Default value = 'default val')\n:type param2: str\n:returns: some value\n:raises keyError: raises key exception\n:raises TypeError: raises type exception"

In [78]:
A.__doc__

' \n    a class without any interest\n    '

inspect.getdoc(object)
Get the documentation string for an object, cleaned up with cleandoc(). If the documentation string for an object is not provided and the object is a class, a method, a property or a descriptor, retrieve the documentation string from the inheritance hierarchy.

Changed in version 3.5: Documentation strings are now inherited if not overridden.

inspect.getcomments(object)
Return in a single string any lines of comments immediately preceding the object’s source code (for a class, function, or method), or at the top of the Python source file (if the object is a module). If the object’s source code is unavailable, return None. This could happen if the object has been defined in C or the interactive shell.

# Loop through all functions and classes

In [49]:
import ast

In [50]:
ast_filename = '/Users/sylvain/Desktop/testpython.py'
with open(ast_filename) as fd:
    file_contents = fd.read()

module = ast.parse(file_contents)
function_definitions = [node for node in module.body if isinstance(node, ast.FunctionDef)]

In [51]:
function_definitions

[<_ast.FunctionDef at 0x7ffb44b6b970>, <_ast.FunctionDef at 0x7ffb44b6bb80>]

In [52]:
[f.name for f in function_definitions]

['func', 'myfunc']

In [79]:
for f in function_definitions:
    print('---')
    print(f.name)
    print('---')
    print(ast.get_docstring(f))

---
func
---
Description of func with docstring groups style (Googledoc).

:param param1: descr of param1 that has True for default value
:param param2: descr of param2 (Default value = 'default val')
:type param2: str
:returns: some value
:raises keyError: raises key exception
:raises TypeError: raises type exception

.. code-block:: python

    def func(x):
        return 2*x
---
myfunc
---
Description of func with docstring groups style (Googledoc).

:param param1: descr of param1 that has True for default value
:param param2: descr of param2 (Default value = 'default val')
:type param2: str
:returns: some value
:raises keyError: raises key exception
:raises TypeError: raises type exception

.. code-block:: python

    def func(x):
        return 2*x


In [80]:
class_definitions = [node for node in module.body if isinstance(node, ast.ClassDef)]
method_definitions = []

for class_def in class_definitions:
    method_definitions.append([node for node in class_def.body if isinstance(node, ast.FunctionDef)])

In [81]:
method_definitions

[[<_ast.FunctionDef at 0x7ffb46a54b80>]]

In [82]:
for m in method_definitions:
    for f in m:
        print('---')
        print(f.name)
        print('---')
        print(ast.get_docstring(f))

---
method
---
:param param1:
:param param2:  (Default value = None)
:rtype: int


In [83]:
ast.get_docstring(class_definitions[0])

''

In [84]:
ast.get_docstring(class_definitions[0].body[1])

':param param1:\n:param param2:  (Default value = None)\n:rtype: int'

# Second pass

In [85]:
import ast

In [89]:
ast.dump(ast.parse("x,y=42"))

"Module(body=[Assign(targets=[Tuple(elts=[Name(id='x', ctx=Store()), Name(id='y', ctx=Store())], ctx=Store())], value=Constant(value=42, kind=None), type_comment=None)], type_ignores=[])"

In [90]:
compile(ast.parse("x=42"),'<input>','exec')

<code object <module> at 0x7ffb46ace9d0, file "<input>", line 1>

In [92]:
eval(compile(ast.parse("x=42"),'<input>','exec'))

In [93]:
x

42

In [110]:
helo_world=ast.Str(s='hello world!',lineno=1,col_offset=1)

In [111]:
print_name=ast.Name(id='print',ctx=ast.Load(),lineno=1, col_offset=1)
print_call=ast.Call(func=print_name,ctx=ast.Load(),args=[helo_world],keywords=[],lineno=1,col_offset=1)
module=ast.Module(body=[ast.Expr(print_call,lineno=1,col_offset=1)],lineno=1,col_offset=1, type_ignores=[])

In [112]:
code=compile(module,'','exec')

In [113]:
eval(code)

hello world!
